### auxfunc.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains all the auxiliary functions used in the other .ipynb files of the repository

In [13]:
# IMPORTS
import import_ipynb
import constants
import pandas as pd
import numpy as np
import pprint
import math

In [14]:
def read_data(anchors, directory_path):
    """
    Reads the csv-files for one specified measurement case and adds them into a dict of dataframes.
    
    Parameters:
    anchors (list): List of anchors, for example ["Anchor 1", "Anchor 2", "Anchor 3"...]
    directory path (string): The relative directory path of the folder which contains the wanted anchor-csv.files.

    returns: 
    scenario_dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.
    """
    scenario_dataframes = {}

    for anchor in anchors:
        current_df = pd.read_csv(directory_path + anchor + ".csv", sep=';')
        scenario_dataframes[anchor.replace(".csv","")] = current_df
    
    return scenario_dataframes

In [15]:
def calculate_smallest_dataset(dataframes):
    """
    Calculates the minimum over all dataframes' length.

    Parameters:
    dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.

    Returns:
    minimum_length (int): an integer which value represents the minimum over all dataframes' length.
    """

    sizes = []

    for anchor in dataframes:
        df = dataframes[anchor]
        sizes.append(len(df))

    return min(sizes)

In [16]:
def calculate_burst_quantity(length):
    """
    Calculates the amount of bursts over all dataframes.

    Parameters:
    length (int): An integer which value represents the number of data to read in the dataset.

    Returns:
    bursts (int): An integer which value represents the amount of bursts / configurations.
    """

    bursts = length // constants.BURST_SIZE
    end = constants.BURST_SIZE * bursts

    if (end < length):
        bursts = bursts + 1
    
    return bursts

In [17]:
def calculate_average_RSS(scenario_dataframe, length):
    """
    Calculates the average RSS value over a burst of 'constants.BURST_SIZE' consecutive measurements.
    This is the implementation of (3) on the reference paper.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    length (int): An integer which value represents the number of data to read in the dataset.

    Returns:
    mean_values (list): Returns a list of average RSS values.
    """
    
    mean_values = []
    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    bursts = calculate_burst_quantity(length)

    start = 0
    for i in range(bursts):
        end = start + constants.BURST_SIZE
        # print("start: ", start)
        if (end <= length):
            # print("end: ", end)
            mean_values.append(round(np.mean(power_values[start:end]), 3))
        else:
            # print("end: ", length)
            mean_values.append(round(np.mean(power_values[start:length]), 3))
        start = end

    return mean_values